In [1]:
# Demo1

import os
import sys

PROJ_DIR = "/Users/woosubs/Desktop/AutomateAnnotation/AnnotationRecommender/"
MOD_DIR = os.path.join(PROJ_DIR, "annotation_recommender")
sys.path.append(MOD_DIR)
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

from annotation_recommender import species_annotation as sa
from annotation_recommender import reaction_annotation as ra
from annotation_recommender import constants as cn
from annotation_recommender import iterator as it
from annotation_recommender import tools

In [11]:
# Create SpeciesAnnotation & ReactionAnnotation class instances
species_an = sa.SpeciesAnnotation(libsbml_fpath=ecoli_fpath)
reaction_an = ra.ReactionAnnotation(libsbml_fpath=ecoli_fpath)

In [12]:
# Identify species IDs to predict
one_rid = 'R_PFK'
comps = reaction_an.getReactionComponents(one_rid)
print("Species associated with reaction %s:" % one_rid, comps)

Species associated with reaction R_PFK: ['M_adp_c', 'M_fdp_c', 'M_f6p_c', 'M_h_c', 'M_atp_c']


In [13]:
# Predict species annotations...
pred_species = species_an.predictAnnotationByName(inp_spec_list=comps)
# for one_spec in species_an.candidates.keys():
#   print("Predicted annotation of %s ->" % one_spec, species_an.candidates[one_spec])
print(species_an.match_score)

{'M_adp_c': 0.3529411764705882, 'M_fdp_c': 1.0, 'M_f6p_c': 1.0, 'M_h_c': 1.0, 'M_atp_c': 0.4117647058823529}


In [14]:
print(species_an.formula)

{'M_adp_c': ['C27O5', 'C16F3IN2O4', 'C14N2O', 'C26FIN5O4', 'C16NO6', 'C21ClN3O2', 'C18N4O11', 'C28N6OS', 'C8NO6', 'C10N5O10P2', 'C29N6O4S', 'C101N7O75', 'C17Cl2F3N7O2S', 'C115N8O85', 'C20O4', 'C12', 'C26N7O2S', 'C19O9P', 'C19N2O2S', 'C18O2', 'C35N4O4', 'C19O2', 'C20N2O5S', 'C9N4O5', 'C18ClN2O6S2'], 'M_fdp_c': ['C6O12P2'], 'M_f6p_c': ['C6O9P'], 'M_h_c': ['H'], 'M_atp_c': ['C20O4', 'C18ClN2O6S2']}


In [15]:
species_an.predictAnnotationByEditDistance(species_an.model.getSpecies('M_fdp_c').name.lower())

{'match_score': 1.0,
 'chebi': ['CHEBI:16905', 'CHEBI:49299'],
 'formula': ['C6O12P2']}

In [16]:
# Predict reaction annotations...
pred_reaction = reaction_an.predictAnnotation(inp_spec_dict=species_an.formula,
                                              inp_reac_list=[one_rid])
print(reaction_an.candidates)
print(reaction_an.match_score)

{'R_PFK': Index(['RHEA:12423', 'RHEA:13380', 'RHEA:14216', 'RHEA:15656', 'RHEA:16112',
       'RHEA:20108'],
      dtype='object')}
{'R_PFK': {'RHEA:12423': 0.8, 'RHEA:13380': 0.8, 'RHEA:14216': 0.8, 'RHEA:15656': 0.8, 'RHEA:16112': 0.8, 'RHEA:20108': 0.8}}


In [17]:
print("Species Accuracy: %.02f" % species_an.getAccuracy())
print("Reaction Accuracy: %.02f" % reaction_an.getAccuracy())

Species Accuracy: 0.80
Reaction Accuracy: 1.00


In [18]:
# Update and predict reaction annotations again
chebi2update = reaction_an.updateSpeciesByAReaction(inp_rid=one_rid, inp_spec_dict=species_an.formula,
                                                    inp_rhea='RHEA:12423')
print(chebi2update)
print()
formula2update = {one_k:[ra.ref_shortened_chebi_to_formula[val] for val in chebi2update[one_k]] \
                  for one_k in chebi2update.keys()}
species_an.candidates.update(chebi2update)
species_an.formula.update(formula2update)
new_pred_annotations = reaction_an.predictAnnotation(inp_spec_dict=species_an.formula,
                                                     inp_reac_list=[one_rid])
print(reaction_an.candidates)
print(reaction_an.match_score)

{'M_atp_c': ['CHEBI:30616']}

{'R_PFK': Index(['RHEA:12423', 'RHEA:13380', 'RHEA:14216', 'RHEA:15656', 'RHEA:16112'], dtype='object')}
{'R_PFK': {'RHEA:12423': 1.0, 'RHEA:13380': 1.0, 'RHEA:14216': 1.0, 'RHEA:15656': 1.0, 'RHEA:16112': 1.0}}


In [8]:
print("Species Accuracy: %.02f" % species_an.getAccuracy())
print("Reaction Accuracy: %.02f" % reaction_an.getAccuracy())

Species Accuracy: 1.00
Reaction Accuracy: 1.00


In [9]:
species_an.predictAnnotationByName(inp_spec_list=None,
                                   specnames_dict={'a': 'Creatine'},
                                   update=False)

{'a': {'match_score': 1.0,
  'chebi': ['CHEBI:16919', 'CHEBI:57947'],
  'formula': ['C4N3O2']}}